# IMPORTS

In [23]:
import pandas as pd
import numpy as np
import os
from decimal import Decimal
import pickle
from pathlib import Path

from sklearn import linear_model

# SETUP

In [24]:
dir_tree_util_path = os.path.join("utils", "dir_tree.py")
exec(open(dir_tree_util_path).read())

# INPUTS

In [113]:
# Paths
path_dados = PROJECT_DIRS["DADOS_DERIVADOS_DIR"]
# path_output = TODO

In [114]:
periodos = [[2010, 2023], [2016, 2023]]

# CARREGANDO p/MEMORIA OS DADOS

In [115]:
# Dados de tráfego ANTT
df_veqs_comercial = pd.read_parquet(path_dados / 'df_VEQS_COMERCIAL.parquet')
df_veqs_passeio = pd.read_parquet(path_dados / 'df_VEQS_PASSEIO.parquet')

dict_veqs = {'df_veqs_comercial':df_veqs_comercial,
           'df_veqs_passeio':df_veqs_passeio}

In [139]:
# Dados de PIB
df_PIB = pd.read_parquet(path_dados / 'PIB-Bacen.parquet')

# TRATANDO OS DADOS

In [140]:
# PIB
df_PIB.index = df_PIB['ano']
df_PIB = df_PIB['cresc_real_ano']
df_PIB = (1 + df_PIB/100)

In [205]:
def filter_PIB_periodo(df_PIB, periodos:list):
    
    # dict_dfs_filtradas_periodo = {f'{p[0]}-{p[1]}':None for p in periodos}
    dict_dfs_filtradas_periodo = {}
    
    for p in periodos:
        ano_min = min(p)
        ano_max = max(p)
        anos = [ano for ano in range(ano_min, ano_max + 1)]

        start_date = pd.to_datetime(f'{p[0]}-01-01').date()
        end_date = pd.to_datetime(f'{p[1]}-12-31').date()
        dict_dfs_filtradas_periodo[f'{p[0]}-{p[1]}'] = df_PIB.loc[anos]
        
    return dict_dfs_filtradas_periodo

In [221]:
def filter_trafego_periodo(dict_trafego:dict, periodos:list):
    dict_dfs_filtradas_periodo = {}
    
    for p in periodos:
        ano_min = min(p)
        ano_max = max(p)
        anos = [str(ano) for ano in range(ano_min, ano_max + 1)]
        
        start_date = pd.to_datetime(f'{p[0]}-01-01').date()
        end_date = pd.to_datetime(f'{p[1]}-12-31').date()
        
        for key, df_veq in dict_veqs.items():
            condition = (df_veq['data_inicial'] <= start_date) \
            & (df_veq['data_final'] > end_date)
            
            dict_veqs[key] = df_veq[condition]
            
    return dict_veqs

In [223]:
filter_trafego_periodo(dict_veqs, periodos)['df_veqs_comercial']

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,data_inicial,data_final,CAGR2010-2023_%,CAGR2016-2023_%
concessionaria,,,,,,,,,,,,,,,,,,,
AUTOPISTA FERNÃO DIAS,92215435.0,111335578.0,115803681.0,117936734.0,114170690.0,100205946.0,95414669.0,99033693.0,100147329.0,120213584.0,125148920.0,141129080.0,114096463.0,115839580.0,7896548.0,2010-01-01,2024-01-31,1.7699330570856597,2.809807646163187
AUTOPISTA FLUMINENSE,23879336.0,26387101.0,27188502.0,26658829.0,25867004.0,23157283.0,20074706.0,18511070.0,18344987.0,19389701.0,20228759.0,24022277.0,23980549.0,24437037.0,4827354.0,2010-01-01,2024-04-16,0.17774554069385218,2.848960718545368
AUTOPISTA LITORAL SUL,69931704.0,74189265.0,77030428.0,78950113.0,98370521.0,85115093.0,79373266.0,74168422.0,75195173.0,79392131.0,81524063.0,91935103.0,90762688.0,95983771.0,30143618.0,2010-01-01,2024-04-30,2.465755368620459,2.7517150889970754
AUTOPISTA PLANALTO SUL,19554781.0,20394485.0,19941264.0,20865630.0,21843099.0,18344124.0,17293967.0,17118560.0,18010664.0,18899658.0,19558701.0,22673561.0,23807238.0,24238904.0,8793128.0,2010-01-01,2024-04-30,1.6655578125923665,4.941067805971455
AUTOPISTA REGIS BITTENCOURT,114884661.0,122893798.0,123630694.0,126658897.0,123633793.0,109082995.0,102808795.0,107596803.0,110472028.0,115068594.0,117259008.0,131519823.0,128857376.0,130655707.0,11173598.0,2010-01-01,2024-02-01,0.9944272399593279,3.48351201498891
CONCER,13109979.0,14804816.0,15723249.0,16066640.0,15291939.0,12496258.0,10926452.0,10202246.0,10505250.0,11027058.0,10553578.0,11169407.0,11038138.0,11360458.0,4901205.0,2010-01-01,2024-06-01,-1.09575976856372,0.5580095493605608
ECOSUL,14426940.0,16168263.0,15672205.0,19186163.0,19934187.0,19358373.0,18450658.0,18671702.0,18907997.0,18599182.0,18051918.0,21509301.0,17553412.0,20311687.0,8416076.0,2010-01-01,2024-06-02,2.6664623954377076,1.3822719725988142
RODOVIA DO AÇO,12887198.0,12595032.0,11550846.0,12922325.0,12850352.0,11591431.0,10991740.0,11594869.0,11896078.0,11944904.0,11674995.0,12551186.0,12456415.0,12951613.0,4229302.0,2010-01-01,2024-05-03,0.03836059321595808,2.3716328164889555
TRANSBRASILIANA,15514995.0,16821634.0,18435974.0,18752871.0,18614344.0,16416821.0,15616264.0,14865436.0,14473693.0,15114298.0,15788230.0,16278144.0,16872695.0,16426187.0,1236821.0,2010-01-01,2024-04-30,0.43996388870104486,0.7249565264233926


In [111]:
# ANTT
cols = [str(ano) for ano in range(ano_min, ano_max+1)]
for df in dict_veqs:
    dict_veqs[df] = dict_veqs[df][cols]

In [128]:

    
    
    # PIB
    x_PIB_p = x_PIB[anos]
    
    for df in dict_veqs:
        # filtrar só as que possuem anos completos no período
        df_p = 

2010-01-01
2023-12-31
2016-01-01
2023-12-31


In [125]:
x_PIB_p

ano
2016    0.9672
2017    1.0132
2018    1.0178
2019    1.0122
2020    0.9672
2021    1.0476
2022    1.0302
2023    1.0291
Name: cresc_real_ano, dtype: float64

In [126]:
df_veqs_comercial

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,data_inicial,data_final,CAGR2010-2023_%,CAGR2016-2023_%
concessionaria,,,,,,,,,,,,,,,,,,,
AUTOPISTA FERNÃO DIAS,92215435.0,111335578.0,115803681.0,117936734.0,114170690.0,100205946.0,95414669.0,99033693.0,100147329.0,120213584.0,125148920.0,141129080.0,114096463.0,115839580.0,7896548.0,2010-01-01,2024-01-31,1.7699330570856597,2.809807646163187
AUTOPISTA FLUMINENSE,23879336.0,26387101.0,27188502.0,26658829.0,25867004.0,23157283.0,20074706.0,18511070.0,18344987.0,19389701.0,20228759.0,24022277.0,23980549.0,24437037.0,4827354.0,2010-01-01,2024-04-16,0.17774554069385218,2.848960718545368
AUTOPISTA LITORAL SUL,69931704.0,74189265.0,77030428.0,78950113.0,98370521.0,85115093.0,79373266.0,74168422.0,75195173.0,79392131.0,81524063.0,91935103.0,90762688.0,95983771.0,30143618.0,2010-01-01,2024-04-30,2.465755368620459,2.7517150889970754
AUTOPISTA PLANALTO SUL,19554781.0,20394485.0,19941264.0,20865630.0,21843099.0,18344124.0,17293967.0,17118560.0,18010664.0,18899658.0,19558701.0,22673561.0,23807238.0,24238904.0,8793128.0,2010-01-01,2024-04-30,1.6655578125923665,4.941067805971455
AUTOPISTA REGIS BITTENCOURT,114884661.0,122893798.0,123630694.0,126658897.0,123633793.0,109082995.0,102808795.0,107596803.0,110472028.0,115068594.0,117259008.0,131519823.0,128857376.0,130655707.0,11173598.0,2010-01-01,2024-02-01,0.9944272399593279,3.48351201498891
CONCEBRA,NaN,NaN,NaN,NaN,NaN,29699543.0,55486986.0,55605633.0,55155048.0,57482064.0,61113709.0,66536252.0,64497601.0,65347707.0,32244565.0,2015-06-01,2024-06-01,N/D,2.3642855749605562
CONCEPA,14096537.0,14804840.0,15742320.0,17190745.0,17876501.0,16530920.0,15943266.0,16238958.0,8102289.0,NaN,NaN,NaN,NaN,NaN,NaN,2010-01-01,2018-07-01,N/D,N/D
CONCER,13109979.0,14804816.0,15723249.0,16066640.0,15291939.0,12496258.0,10926452.0,10202246.0,10505250.0,11027058.0,10553578.0,11169407.0,11038138.0,11360458.0,4901205.0,2010-01-01,2024-06-01,-1.09575976856372,0.5580095493605608
CRO,NaN,NaN,NaN,NaN,NaN,26619773.0,78080309.0,75985903.0,81527999.0,82205042.0,85291561.0,93454156.0,98050077.0,103036868.0,35774523.0,2015-09-01,2024-06-03,N/D,4.0416673912228696


In [107]:
df_veqs_comercial[cols].T

concessionaria,AUTOPISTA FERNÃO DIAS,AUTOPISTA FLUMINENSE,AUTOPISTA LITORAL SUL,AUTOPISTA PLANALTO SUL,AUTOPISTA REGIS BITTENCOURT,CONCEBRA,CONCEPA,CONCER,CRO,CRT,...,MSVIA,NOVADUTRA,RIOSP,RODOVIA DO AÇO,TRANSBRASILIANA,VIA 040,VIA BAHIA,VIA BRASIL,VIA COSTEIRA,VIA SUL
2010,92215435.0,23879336.0,69931704.0,19554781.0,114884661.0,NaN,14096537.0,13109979.0,NaN,8563340.0,...,NaN,116380982.0,NaN,12887198.0,15514995.0,NaN,NaN,NaN,NaN,NaN
2011,111335578.0,26387101.0,74189265.0,20394485.0,122893798.0,NaN,14804840.0,14804816.0,NaN,9264103.0,...,NaN,125958069.0,NaN,12595032.0,16821634.0,NaN,NaN,NaN,NaN,NaN
2012,115803681.0,27188502.0,77030428.0,19941264.0,123630694.0,NaN,15742320.0,15723249.0,NaN,9892007.0,...,NaN,122928827.0,NaN,11550846.0,18435974.0,NaN,NaN,NaN,NaN,NaN
2013,117936734.0,26658829.0,78950113.0,20865630.0,126658897.0,NaN,17190745.0,16066640.0,NaN,9813665.0,...,NaN,123410424.0,NaN,12922325.0,18752871.0,NaN,67484476.0,NaN,NaN,NaN
2014,114170690.0,25867004.0,98370521.0,21843099.0,123633793.0,NaN,17876501.0,15291939.0,NaN,10186433.0,...,NaN,119182763.0,NaN,12850352.0,18614344.0,NaN,69684068.0,NaN,NaN,NaN
2015,100205946.0,23157283.0,85115093.0,18344124.0,109082995.0,29699543.0,16530920.0,12496258.0,26619773.0,9494459.0,...,11836118.0,103840572.0,NaN,11591431.0,16416821.0,21317801.0,63954816.0,NaN,NaN,NaN
2016,95414669.0,20074706.0,79373266.0,17293967.0,102808795.0,55486986.0,15943266.0,10926452.0,78080309.0,8708634.0,...,37685960.0,96135285.0,NaN,10991740.0,15616264.0,43724033.0,58857066.0,NaN,NaN,NaN
2017,99033693.0,18511070.0,74168422.0,17118560.0,107596803.0,55605633.0,16238958.0,10202246.0,75985903.0,7681400.0,...,33063928.0,96841380.0,NaN,11594869.0,14865436.0,44369281.0,61468404.0,NaN,NaN,NaN
2018,100147329.0,18344987.0,75195173.0,18010664.0,110472028.0,55155048.0,8102289.0,10505250.0,81527999.0,7692691.0,...,34911994.0,96333329.0,NaN,11896078.0,14473693.0,46504247.0,61710215.0,NaN,NaN,NaN
2019,120213584.0,19389701.0,79392131.0,18899658.0,115068594.0,57482064.0,NaN,11027058.0,82205042.0,7823780.0,...,34303270.0,98070770.0,NaN,11944904.0,15114298.0,49583988.0,62116167.0,NaN,NaN,11719433.0
